In [ ]:
# sources that helped us make our model:
# 1.https://lajavaness.medium.com/multiclass-and-multilabel-text-classification-in-one-bert-model-95c54aab59dc
# helped us figure out a method to do multilabel - multiclass classification while using hugging face
# especially helpful with regards to creating a custom loss function
# 2. https://huggingface.co/learn/nlp-course/chapter1/1
# taught us general information regarding hugging face and how to fine tune our own model using an LLM like BERT

In [ ]:
!pip install datasets
!pip install accelerate -U
!pip install scikit-multilearn

In [ ]:
import csv
import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection.iterative_stratification import iterative_train_test_split as ITTS

In [ ]:
data = []
finalData = [["Text", "Palestine Sympathy Rating", "Israel Sympathy Rating", "Palestine Military Rating", "Israel Military Rating"]]
X = []
y = []

In [ ]:
with open("/content/finalAllData.csv", newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
ALL_LABELS = {
"PS0": "negtive palestine sympathy",
"PS1": "neutral palestine sympathy",
"PS2": "positive palestine sympathy",
"IS0": "negtive israel  sympathy",
"IS1": "neutral israel  sympathy",
"IS2": "positive israel sympathy",
"PM0": "negative palestine military",
"PM1": "neutral palestine military",
"PM2": "positive palestine military",
"IM0": "negative israel military",
"IM1": "neutral israel military",
"IM2": "positive israel military"
}

In [ ]:
def redefine(ls):
    #-1, 0, and 1
    for i in range(4):
        if ls[i].strip() == "+":
            ls[i] = 1.0
        elif ls[i].strip() == "-":
            ls[i] = -1.0
        elif ls[i].lower().strip() == "n":
            ls[i] = 0.0

    return ls

In [ ]:
for z in range(1, len(data)):
    x = data[z]
    X.append([x[0]])

    ls = x[1:]
    ls = redefine(ls)
    y.append(ls)
    _ = []
    _.append(x[0])
    _.append(ls)

    finalData.append(_)

print(len(X))

In [ ]:
with open("/content/test_data.csv", newline='') as f:
    reader = csv.reader(f)
    testData = list(reader)

In [ ]:
X_test = []
y_test = []


for z in range(1, len(testData)):
    x = testData[z]
    X_test.append([x[0]])

    ls = x[1:]
    ls = redefine(ls)
    y_test.append(ls)

In [ ]:
def ratingDisplay(providedList):
    ratingsDict = {}
    for ls in providedList:
        if str(ls) in ratingsDict:
            ratingsDict[str(ls)] += 1
        else:
            ratingsDict[str(ls)] = 1

    for key in ratingsDict:
        print(key + ": " + str(ratingsDict[key]))
    print(sum(ratingsDict.values()))

In [ ]:
deleted = []

In [ ]:
X_test[15] = ['"There is no valid reason to believe that Palestinian claim to the land is "finished". The Palestinian expectation has never been that the Israelis give up "everything". The expectation has been to stop the colonization, and allow Palestinian self-rule over their land."']
X_test[33] = ['"The problem with that argument is that depending on which poll Hamas already has between 60% and 80% of open and enthusiastic support of Palestinians. People love to make excuses for the Palestinian Population not being Hamas, but the fact is most share their worldview and are willing to follow Hamas and fight, rape, torture and murder "the jews"."']
X_test[34] = ['''"The videos of 7th October that Hamas launched shows Palestinian civilians including kids taking Israeli women as hostages. Why didn't Palestinians stick to the two state solution when Arafat signed the peace treaty? And why do Palestinians claim it as their land when everyone knows that it's an ancestral jew land? Remember, it wasn't Hamas with all those suicide bombings in Israel all throughout the creation of "new Israel", it was Palestinian civilians."''']
X_test[55] = ['''"The premise of Israel's legitimacy doesn't make sense. You call another people's land halfway across the world which you know nothing your "ancestral homeland". You play the "historical persecution" to justify your existence. You terrorize and genocide the local indigenous population and try to push then out. You rely of systematic oppression and murder for your security. You take over their culture and rename it in your language, which is actually your second."''']
X_test[65] = ['"You portray Palestinians as being too proud to "surrender to peace." Surrendering to a life of indignity under Israeli blockade, where basic human needs and rights are denied and there is no legal protection from the excesses of the occupation. Are you expecting them to silently accept the building of Israeli settlements over their land."']

for x in X_test:
    bob = True
    w = x[0]
    w = w[1:-1]
    for z in X:
        if w in z[0]:
            index = X.index(z)
            del X[index]
            del y[index]
            bob = False
            deleted.append(x)
    if bob and x not in deleted:
        print(X_test.index(x))
        print(x[0])

print(len(X))

In [ ]:
y_sympathy_p = []
y_sympathy_i = []
y_military_p = []
y_military_i = []

for dataPoint in y:
  y_sympathy_p.append(dataPoint[0])
  y_sympathy_i.append(dataPoint[1])
  y_military_p.append(dataPoint[2])
  y_military_i.append(dataPoint[3])

ratingDisplay(y_sympathy_p)
ratingDisplay(y_sympathy_i)
ratingDisplay(y_military_p)
ratingDisplay(y_military_i)

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test).astype(float)

X = np.array(X)
y = np.array(y).astype(float)

In [ ]:
X_train, y_train, X_validation, y_validation = ITTS(X, y, test_size=0.125)

X_validation = np.array(X_validation)
y_validation = np.array(y_validation).astype(float)

In [ ]:
correlations = np.corrcoef(y.T)
correlations

In [ ]:
# Palestine Sympathy--Israel Sympathy--Palestine Military--Israel Military
#create a new figure
cols = ["PSympathy", "ISympathy", "PMilitary", "IMilitary"]
fig = plt.figure(figsize=(10,10))

#111: 1x1 grid, first subplot
ax = fig.add_subplot(111)

#normalize data using vmin, vmax
cax = ax.matshow(correlations, vmin=-1, vmax=1)

#add a colorbar to a plot.
fig.colorbar(cax)

#define ticks
ticks = np.arange(0,4)

#set x and y tick marks
ax.set_xticks(ticks)
ax.set_yticks(ticks)

#set x and y tick labels
ax.set_xticklabels(cols)
ax.set_yticklabels(cols)

#draw a matrix using the correlations data
plt.show()

In [ ]:
print("train set:")
# ratingDisplay(y_train)

y_sympathy_p_train = []
y_sympathy_i_train = []
y_military_p_train = []
y_military_i_train = []

for dataPoint in y_train:
  if dataPoint[0] == -1.:
    y_sympathy_p_train.append("PS0")
  elif dataPoint[0] == 0.:
    y_sympathy_p_train.append("PS1")
  else:
    y_sympathy_p_train.append("PS2")
  if dataPoint[1] == -1.:
    y_sympathy_i_train.append("IS0")
  elif dataPoint[1] == 0.:
    y_sympathy_i_train.append("IS1")
  else:
    y_sympathy_i_train.append("IS2")
  if dataPoint[2] == -1.:
    y_military_p_train.append("PM0")
  elif dataPoint[2] == 0.:
    y_military_p_train.append("PM1")
  else:
    y_military_p_train.append("PM2")
  if dataPoint[3] == -1.:
    y_military_i_train.append("IM0")
  elif dataPoint[3] == 0.:
    y_military_i_train.append("IM1")
  else:
    y_military_i_train.append("IM2")

print("\nvalidation set:")
# ratingDisplay(y_validation)

y_sympathy_p_val = []
y_sympathy_i_val = []
y_military_p_val = []
y_military_i_val = []

for dataPoint in y_validation:
  if dataPoint[0] == -1.:
    y_sympathy_p_val.append("PS0")
  elif dataPoint[0] == 0.:
    y_sympathy_p_val.append("PS1")
  else:
    y_sympathy_p_val.append("PS2")
  if dataPoint[1] == -1.:
    y_sympathy_i_val.append("IS0")
  elif dataPoint[1] == 0.:
    y_sympathy_i_val.append("IS1")
  else:
    y_sympathy_i_val.append("IS2")
  if dataPoint[2] == -1.:
    y_military_p_val.append("PM0")
  elif dataPoint[2] == 0.:
    y_military_p_val.append("PM1")
  else:
    y_military_p_val.append("PM2")
  if dataPoint[3] == -1.:
    y_military_i_val.append("IM0")
  elif dataPoint[3] == 0.:
    y_military_i_val.append("IM1")
  else:
    y_military_i_val.append("IM2")


print("\ntest set:")
# ratingDisplay(y_test)

y_sympathy_p_test = []
y_sympathy_i_test = []
y_military_p_test = []
y_military_i_test = []

for dataPoint in y_test:
  if dataPoint[0] == -1.:
    y_sympathy_p_test.append("PS0")
  elif dataPoint[0] == 0.:
    y_sympathy_p_test.append("PS1")
  else:
    y_sympathy_p_test.append("PS2")
  if dataPoint[1] == -1.:
    y_sympathy_i_test.append("IS0")
  elif dataPoint[1] ==0.:
    y_sympathy_i_test.append("IS1")
  else:
    y_sympathy_i_test.append("IS2")
  if dataPoint[2] == -1.:
    y_military_p_test.append("PM0")
  elif dataPoint[2] == 0.:
    y_military_p_test.append("PM1")
  else:
    y_military_p_test.append("PM2")
  if dataPoint[3] == -1.:
    y_military_i_test.append("IM0")
  elif dataPoint[3] == 0.:
    y_military_i_test.append("IM1")
  else:
    y_military_i_test.append("IM2")

In [ ]:
import numpy as np
import torch

from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

CHECKPOINT = "bert-base-uncased"
MAX_LENGTH = 512
BATCH_SIZE = 4
EPOCHS = 6

id2label = {k:v for k, v in enumerate(ALL_LABELS)}
label2id = {v:k for k, v in enumerate(ALL_LABELS)}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, id2label=id2label, label2id=label2id)

In [ ]:
# credit: https://lajavaness.medium.com/multiclass-and-multilabel-text-classification-in-one-bert-model-95c54aab59dc
# function for turning our old labels (4 long) to our new array of labels (12 long) that our model can use
def preprocess_function(examples: dict):
    labels = [0] * len(id2label)
    for k, l in id2label.items():
        if l == examples["palestine_sympathy"] or l == examples["israel_sympathy"] or l == examples["palestine_military"] or l == examples["israel_military"]:
            labels[k] = 1
        else:
            labels[k] = 0
    examples = tokenizer(examples["text"])
    examples["label"] = labels
    return examples

In [ ]:
from datasets import Dataset

X_tr = []
for i in range(len(X_train)):
    X_tr.append(X_train[i][0])

train_dict = {}
train_dict["text"] = X_tr
train_dict["palestine_sympathy"] = y_sympathy_p_train
train_dict["israel_sympathy"] = y_sympathy_i_train
train_dict["palestine_military"] = y_military_p_train
train_dict["israel_military"] = y_military_i_train
train_ds = Dataset.from_dict(train_dict)

In [ ]:
X_val = []
for i in range(len(X_validation)):
    X_val.append(X_validation[i][0])

val_dict = {}
val_dict["text"] = X_val
val_dict["palestine_sympathy"] = y_sympathy_p_val
val_dict["israel_sympathy"] = y_sympathy_i_val
val_dict["palestine_military"] = y_military_p_val
val_dict["israel_military"] = y_military_i_val
val_ds = Dataset.from_dict(val_dict)

In [ ]:
X_tst = []
for i in range(len(X_test)):
    X_tst.append(X_test[i][0])

test_dict = {}
test_dict["text"] = X_tst
test_dict["palestine_sympathy"] = y_sympathy_p_test
test_dict["israel_sympathy"] = y_sympathy_i_test
test_dict["palestine_military"] = y_military_p_test
test_dict["israel_military"] = y_military_i_test
test_ds = Dataset.from_dict(test_dict)

In [ ]:
ds = {"train": train_ds, "validation": val_ds, "test": test_ds}
for dset in ds:
    ds[dset] = ds[dset].map(preprocess_function, remove_columns=["text", "palestine_sympathy", "israel_sympathy", "palestine_military", "israel_military"])

In [ ]:
PS_LABELS = ["PS0", "PS1", "PS2"]
PS_INDICES = range(3)
IS_LABELS = ["IS0", "IS1", "IS2"]
IS_INDICES = range(3,6)
PM_LABELS = ["PM0", "PM1", "PM2"]
PM_INDICES = range(6,9)
IM_LABELS = ["IM0", "IM1", "IM2"]
IM_INDICES = range(9,12)

In [ ]:
# credit: https://lajavaness.medium.com/multiclass-and-multilabel-text-classification-in-one-bert-model-95c54aab59dc
# showed a way to find the max argument for individual slices of our labels
def predict(logits):
    ret = np.zeros(logits.shape)

    PS_index = np.argmax(logits[:,PS_INDICES], axis=-1)
    ret[range(len(ret)), PS_index] = 1
    IS_index = np.argmax(logits[:,IS_INDICES], axis=-1) + 3
    ret[range(len(ret)), IS_index] = 1
    PM_index = np.argmax(logits[:,PM_INDICES], axis=-1) + 6
    ret[range(len(ret)), PM_index] = 1
    IM_index = np.argmax(logits[:, IM_INDICES], axis=-1) + 9
    ret[range(len(ret)), IM_index] = 1

    return ret

In [ ]:
# credit: https://lajavaness.medium.com/multiclass-and-multilabel-text-classification-in-one-bert-model-95c54aab59dc
# custom compute metrics function to accomodate for the multiple classes

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    final_metrics = {}

    # get predictions from logits
    predictions = predict(logits)

    # f1 scores per class
    final_metrics["f1_micro_for_PS"] = f1_score(labels[:, PS_INDICES], predictions[:, PS_INDICES], average="micro")
    final_metrics["f1_micro_for_IS"] = f1_score(labels[:, IS_INDICES], predictions[:, IS_INDICES], average="micro")
    final_metrics["f1_micro_for_PM"] = f1_score(labels[:, PM_INDICES], predictions[:, PM_INDICES], average="micro")
    final_metrics["f1_micro_for_IM"] = f1_score(labels[:, IM_INDICES], predictions[:, IM_INDICES], average="micro")
    # global f1 scores
    final_metrics["f1_micro"] = f1_score(labels, predictions, average="micro")
    final_metrics["f1_macro"] = f1_score(labels, predictions, average="macro")
    final_metrics["f1_weighted"] = f1_score(labels, predictions, average="weighted")

    # classification reports
    print("Classification report for PS: ")
    print(classification_report(labels[:, PS_INDICES], predictions[:, PS_INDICES], zero_division=0))
    print("Classification report for IS: ")
    print(classification_report(labels[:, IS_INDICES], predictions[:, IS_INDICES], zero_division=0))
    print("Classification report for PM: ")
    print(classification_report(labels[:, PM_INDICES], predictions[:, PM_INDICES], zero_division=0))
    print("Classification report for IM: ")
    print(classification_report(labels[:, IM_INDICES], predictions[:, IM_INDICES], zero_division=0))
    return final_metrics

In [ ]:
from matplotlib.pylab import plt

# our own custom function to graph the training loss, validation loss, and weighted f1 score after each epoch
def loss_graphing():
    training_loss = []
    validation_loss = []
    weighted_f1 = []
    loss = trainer.state.log_history
    for dic in loss:
      if 'eval_loss' in dic:
        validation_loss.append(dic['eval_loss'])
        weighted_f1.append(dic['eval_f1_weighted'])
      elif 'loss' in dic:
        training_loss.append(dic['loss'])
      else:
        final_loss = dic

    epochs_val = range(1,len(validation_loss)+1)
    epochs_train = range(1,len(training_loss)+1)

    plt.plot(epochs_train, training_loss, label='Training Loss')
    plt.plot(epochs_val, validation_loss, label='Validation Loss')
    plt.plot(epochs_val, weighted_f1, label='Weighted F1')
    plt.axhline(y=0.6, color='r', linestyle='--', linewidth=2)

    plt.legend(loc='best')

    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xticks(np.arange(1, 7, 1))

In [ ]:
# credit: https://lajavaness.medium.com/multiclass-and-multilabel-text-classification-in-one-bert-model-95c54aab59dc
# custom loss function in order to take the imbalance of classes into account

class MultiTaskClassificationTrainer(Trainer):
    def __init__(self, group_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.group_weights = group_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]

        PS_inp = logits[:, 0:3].type(torch.float32)
        PS_lab = labels[:, 0:3].type(torch.float32)
        IS_inp = logits[:, 3:6].type(torch.float32)
        IS_lab = labels[:, 3:6].type(torch.float32)
        PM_inp = logits[:, 6:9].type(torch.float32)
        PM_lab = labels[:, 6:9].type(torch.float32)
        IM_inp = logits[:, 9:12].type(torch.float32)
        IM_lab = labels[:, 9:12].type(torch.float32)

        # our own custom weights - majority of the tweaking between training was done on these
        wPS = torch.Tensor([7.435,1.77,3.17])
        wIS = torch.Tensor([11.475,1.45,4.11])
        wPM = torch.Tensor([2.36,1.87,12.41])
        wIM = torch.Tensor([2.32,2.52,5.58])

        PS_loss = torch.nn.functional.cross_entropy(PS_inp, PS_lab,weight=wPS)
        IS_loss = torch.nn.functional.cross_entropy(IS_inp, IS_lab,weight=wIS)
        PM_loss = torch.nn.functional.cross_entropy(PM_inp, PM_lab,weight=wPM)
        IM_loss = torch.nn.functional.cross_entropy(IM_inp, IM_lab,weight=wIM)

        loss = self.group_weights[0] * PS_loss + self.group_weights[1] * IS_loss + self.group_weights[2] * PM_loss + self.group_weights[3] * IM_loss
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="./content/oogabooga",
    learning_rate=5e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="f1_macro",
    load_best_model_at_end=False,
    logging_strategy="epoch",
    weight_decay=0.01,
)

In [ ]:
trainer = MultiTaskClassificationTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    group_weights=(0.25, 0.25, 0.25, 0.25)
)

In [ ]:
trainer.train()

In [ ]:
loss_graphing()

In [ ]:
trainer.evaluate(eval_dataset=ds["test"])

In [ ]:
loss_graphing()

In [ ]:
trainer.save_model("/content/bees")

In [ ]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

In [ ]:
labels = ['-', "n", "+"] * 4

def classification(text):
  classified = pipe(text)
  scores = []

  for label in classified[0]:
    scores.append(label["score"])

  scores = np.array(scores)
  print(scores)
  PS = np.argmax(scores[0:3])
  IS = np.argmax(scores[3:6])
  PM = np.argmax(scores[6:9])
  IM = np.argmax(scores[9:12])

  print(np.array(["PS: "+labels[PS], "IS: "+labels[IS], "PM: "+labels[PM], "IM: "+labels[IM]]).reshape(4,1))

In [ ]:
from bs4 import BeautifulSoup
import requests
import sys
import re

#neither work w/ JS, old works only for a few sites
def findingTextNewer(soup):
  paragraphs = (soup.find_all("p"))

  for p in paragraphs:
    depth = 0
    for _ in p.parents:
      depth = depth + 1

    if str(depth) in depths:
      depths[str(depth)] = depths[str(depth)] + len(p.text)
    else:
      depths[str(depth)] = 1

  level = int(max(depths, key=lambda key: depths[key]))
  # while(depths[str(level)] > 100):
  #   del depths[str(level)]
  #   level = int(max(depths, key=lambda key: depths[key]))

  for p in paragraphs:
    d = 0
    for _ in p.parents:
      d = d + 1

    if (d == level):
      INP = p.get_text().strip()
      if INP != "":
        print(INP)
        classification(INP)

#test websites:
#CNN - https://www.cnn.com/2023/12/21/politics/trump-legal-chaos-2024/index.html
#NYT - https://www.nytimes.com/2023/12/20/movies/home-alone-mccallisters-wealth.html
#washington post - https://www.washingtonpost.com/world/2023/12/21/al-shifa-hospital-gaza-hamas-israel/
#cnbc - https://www.cnbc.com/2023/12/21/former-trump-lawyer-rudy-giuliani-files-for-bankruptcy-protection.html
#fox news - https://www.foxnews.com/politics/maine-forced-delay-vote-ev-mandate-amid-widespread-power-outages
#bbc - https://www.bbc.com/news/world-middle-east-67764664
#reuters - https://www.reuters.com/world/middle-east/egypt-seeks-broker-gaza-ceasefire-hamas-israel-assert-demands-2023-12-21/ - not working
#al jazeera - https://www.aljazeera.com/news/2023/12/21/why-is-israels-military-killing-so-many-of-its-own
#nbc - https://www.nbcnews.com/news/world/israels-offensive-hamas-gaza-succeeding-rcna130745
#the guardian - https://www.theguardian.com/world/2023/dec/21/israel-idf-accused-targeting-journalists-gaza

depths = {}

headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"}

url = input("Enter the link to a news article: ")

if matches := re.match(r"^http?(s):\/\/(www\.)?(.+)\.co.+$",url):
  page = requests.get(url, headers=headers)
  scraped = BeautifulSoup(page.content,"html.parser")
else:
  print("invald link")

#https://www.cnn.com/2023/12/21/politics/trump-legal-chaos-2024/index.html

findingTextNewer(scraped)